In [2]:
DF_PATH       = "../data/processed/cleaned_data.pkl"
ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith']

## Import tools

In [16]:
# Load packages
import pandas as pd 
import numpy as np
import yaml
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt


In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)

## Functions

In [12]:
def calculate_quality(ground_truth, prediction, metric_function, sort_values=False):
    quality_scores = {}
    for col in prediction.columns:
        role_pred  = prediction[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    return quality_scores


## Balance classes

In [4]:
roles_df = df.iloc[:, :16].copy()
role_sum = roles_df.sum(axis=0)
role_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [5]:
# Resample roles
samples_per_class = 500
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)


In [6]:
# Construct dfs
roles_df  = pd.concat(resampled_roles)
df = df.loc[roles_df.index]

In [7]:
roles_df.sum(axis=0)

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, 16:], df.iloc[:, :16], random_state=0)

## Random forest with non-linearity

In [9]:
rf_clf = make_pipeline(StandardScaler(), 
                       FeatureUnion([('linear_pca', PCA(n_components=40)), 
                                      ('kernel_pca', KernelPCA(n_components=40, 
                                                               kernel='rbf'))]),
                       RandomForestClassifier(random_state=0))

rf_clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('featureunion',
                 FeatureUnion(transformer_list=[('linear_pca',
                                                 PCA(n_components=40)),
                                                ('kernel_pca',
                                                 KernelPCA(kernel='rbf',
                                                           n_components=40))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [13]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [14]:
train_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,98.83,96.93,91.02,93.88
Data or business analyst,99.33,97.59,95.62,96.59
Data scientist or machine learning specialist,99.23,97.11,95.94,96.52
Database administrator,99.32,94.69,96.18,95.43
DevOps specialist,99.57,99.41,95.64,97.49
"Developer, QA or test",99.40,99.49,92.06,95.63
"Developer, back-end",98.67,98.98,94.94,96.92
"Developer, desktop or enterprise applications",98.68,96.70,91.39,93.97
"Developer, embedded applications or devices",98.78,94.23,91.04,92.60
"Developer, front-end",99.43,97.53,97.53,97.53


## Predict sample entery

In [85]:
prediction = rf_clf.predict_proba(X_test.iloc[:1, :])
positive_probs = [prob[0][1] for prob in prediction]
pd.Series(positive_probs, 
          index=Y_train.columns).sort_values(ascending=False)


Developer, full-stack                            0.755000
Developer, back-end                              0.703881
Developer, front-end                             0.178690
Database administrator                           0.073810
Developer, mobile                                0.054333
DevOps specialist                                0.030000
Developer, QA or test                            0.021250
Data scientist or machine learning specialist    0.020000
System administrator                             0.012500
Data or business analyst                         0.011250
Academic researcher                              0.010000
Developer, embedded applications or devices      0.010000
Developer, game or graphics                      0.010000
Engineer, data                                   0.010000
Developer, desktop or enterprise applications    0.001250
Scientist                                        0.001250
dtype: float64